# Common problems and pitfalls

Regular Vector Fitting is susceptible to some general problems and to specific user errors. This section is intended to collect and address common issues and to help mitigating them. Additional explanations and background information can be found in the [Vector Fitting tutorial](../../tutorials/VectorFitting.ipynb).

## Sources of problems

The following things can cause significant problems in the vector fitting process and may result in poor convergence and/or large errors of the fit:
1. not enough poles
2. too many poles
3. strong noise in the data

From points 1) and 2) it is obvious that the number of poles has to be chosen carefully, which often requires several attempts to get right. But even with the right number of poles, the fit quality and the required number of iterations can be disappointing in case of strong noise in the data, usually from measurements. In such cases, successful vector fits might still be achieved with `auto_fit()`, an improved and automated version of the vector fitting algorithm utilizing iterative pole adding and skimming.

The 2-port measurement from example 2 is a tricky network to fit properly. It will serve as an example of the different issues. Also see [Ex2: Measured 190 GHz Active 2-Port](vectorfitting_ex2_190ghz_active.ipynb) for a demonstration of `auto_fit()` on this network.

In [ ]:
import matplotlib.pyplot as mplt
import numpy as np

import skrf

nw = skrf.network.Network('./190ghz_tx_measured.S2P')
vf = skrf.VectorFitting(nw)
freqs = np.linspace(100e9, 300e9, 401)

## Example of noisy data with not enough poles

Three real poles and one complex conjugate pair is not enough for an acurrate fit of this network. It takes surprisingly many iterations and the resulting model error is still fairly large:

In [ ]:
vf.vector_fit(n_poles_init=5, poles_init_type='real')

In [ ]:
print(f'RMS error = {vf.get_rms_error()}')

In [ ]:
vf.plot_convergence()

In [ ]:
vf.plot_s_db(freqs=freqs)

## Example of noisy data with too many poles

Ten real poles and ten complex conjugate pairs are too much for this simple network. The unsued poles get shifted toward higher frequencies outside the measured band and still spook around during the relocation process. Due to these spurious pole, the relocation process cannot converge properly and gets stopped after reaching the maxmimum number of iterations (default is 200). Interstingly, the residue settles rather quickly within the first 20 iterations, as shown in the convergence plot.
Still, the fitting result within the measured frequency band is very good, but the spurious out-of-band poles are clearly visible.

In [ ]:
vf.vector_fit(n_poles_init=20, poles_init_type='complex')

In [ ]:
vf.plot_convergence()

In [ ]:
print(f'RMS error = {vf.get_rms_error()}')

In [ ]:
vf.plot_s_db(freqs=freqs)

## Comment on starting poles

During the pole relocation process (first step in the fitting process), the starting poles are sucessively moved to frequencies where they can best match all target responses. Additionally, the type of poles can change from real to complex-conjugate: two real poles can become one complex-conjugate pole (and vise versa). As a result, there are multiple combinations of starting poles which can produce the same final set of poles. However, certain setups will converge faster than others, which also depends on the initial pole spacing. In extreme cases, the algorithm can even be "undecided" if two real poles behave exactly like one complex-conjugate pole and it gets "stuck" jumping back and forth without converging to a final solution.

In [ ]:
vf.vector_fit(n_poles_init=5, poles_init_type='complex')
vf.plot_convergence()

Even though the pole relocation process oscillated between two (or more?) solutions and did not converge, the fit was still successful, because the solutions themselves did converge:

In [ ]:
vf.get_rms_error()

In [ ]:
fig, ax = mplt.subplots(2, 2)
fig.set_size_inches(12, 8)
vf.plot_s_mag(0, 0, freqs=freqs, ax=ax[0][0]) # s11
vf.plot_s_mag(0, 1, freqs=freqs, ax=ax[0][1]) # s12
vf.plot_s_mag(1, 0, freqs=freqs, ax=ax[1][0]) # s21
vf.plot_s_mag(1, 1, freqs=freqs, ax=ax[1][1]) # s22
fig.tight_layout()
mplt.show()